In [1]:
%run utils.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def evaluate_models(features,labels,models,scoring_method='accuracy',CV=5):
    cv_df = pd.DataFrame(index=range(CV * len(models)))
    entries = []
    for model in models:
        model_name = model.__class__.__name__
        accuracies = cross_val_score(model,features, labels, scoring=scoring_method, cv=CV)
        for fold_idx, accuracy in enumerate(accuracies):
            entries.append((model_name, fold_idx, accuracy))
    cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', scoring_method])
    sns.boxplot(x='model_name', y=scoring_method, data=cv_df)
    sns.stripplot(x='model_name', y=scoring_method, data=cv_df, 
                  size=8, jitter=True, edgecolor="gray", linewidth=2)
    plt.title('Scoring Method={}'.format(scoring_method))
    plt.xticks(rotation=45)
    a=plt.show()
    print(cv_df.groupby('model_name')[scoring_method].mean())
    return cv_df


def evaluate_model_stats(model,X_train, X_test, y_train, y_test,train=True):
    if train:
        model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_probs=model.predict_proba(X_test)
    conf_mat = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(10,10))
   # sns.heatmap(conf_mat, annot=True, fmt='d',
                #xticklabels=AUTHOR_ID.keys(), yticklabels=AUTHOR_ID.keys())
    sns.heatmap(conf_mat, annot=True, fmt='d')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title("{} Confusion Martix".format(model.__class__.__name__))

    print(metrics.classification_report(y_test, y_pred, target_names=['No-Popular','Popular']))

    #calculate log-loss
    print("{}:Log-Loss:{}".format(model.__class__.__name__,log_loss(y_test,y_pred_probs)))
    return model,y_pred,y_pred_probs,conf_mat
